In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import pyspark.sql.types as TS
from pyspark.sql.functions import dayofweek
from pyspark.sql.functions import monotonically_increasing_id

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
11,application_1672655428168_0012,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://sparkifyggo/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# get filepath to song data file
song_data = '{}{}'.format(input_data, 'song_data/*/*/*/*.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# read song data file
df = spark.read.json(song_data)    # extract columns to create songs table
songs_table = df.select('song_id','title','artist_id','year','duration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
print('Done')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Done

In [7]:
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy('year','artist_id').mode('overwrite').parquet(output_data + "songs/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Error sending http request and maximum retry encountered.


In [8]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [9]:
# extract columns to create artists table
artists_table = df.select('artist_id','artist_name','artist_location',
                                   'artist_latitude','artist_longitude')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [10]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [11]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data + "artists/")

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [12]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [13]:
log_data = '{}{}'.format(input_data, 'log_data/*/*/*.json')
song_data = '{}{}'.format(input_data, 'song_data/*/*/*/*.json')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [14]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [15]:
# read log data file
df = spark.read.json(log_data)

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [16]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [17]:
# filter by actions for song plays
df = df.where(df.page == 'NextSong')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [18]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [19]:
# extract columns for users table    
users_table = df.select('userId','firstName','lastName','gender','level')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [20]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [21]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data + "users/")

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [22]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [23]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x/1000), TS.TimestampType())
df = df.withColumn('ts_timestamp',get_timestamp('ts'))

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [24]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [25]:
# extract columns to create time table
time_table = df.select('ts_timestamp', hour('ts_timestamp').alias('hour'), dayofmonth('ts_timestamp').alias('day'), weekofyear('ts_timestamp').alias('weekofyear'),
                        month('ts_timestamp').alias('month'), year('ts_timestamp').alias('year'), dayofweek('ts_timestamp').alias('dayofweek'))

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [26]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [27]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year', 'month').mode('overwrite').parquet(output_data + "time/")

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [28]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [29]:
# read in song data to use for songplays table
song_df = spark.read.json(song_data)

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [30]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [31]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = df.join(song_df, df['song'] == song_df['title']).drop('title')
songplays_table = songplays_table.withColumn('songplay_id', monotonically_increasing_id()).select('songplay_id','ts_timestamp', 'userId', 'level',
                                                                                                'song_id', 'artist_id', 'sessionId', 'location', 'userAgent')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [32]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [33]:
# create year and month columns for the partition 
songplays_table = songplays_table.withColumn('year', year('ts_timestamp')).withColumn('month', month('ts_timestamp'))

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [34]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [35]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy('year', 'month').mode('overwrite').parquet(output_data + "songplays/")

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.


In [36]:
print('Done')

VBox()

An error was encountered:
Error sending http request and maximum retry encountered.
